### handle .db file (sqlite)

In [2]:
# Import Required Libraries
import sqlite3
import pandas as pd

In [17]:
DB_PATH = 'datasrc/cac-data-contd-32.db'
NEW_DB_PATH = 'new_analysis.db'

In [18]:
# Query definitions
query_read_all_tables_names = "SELECT name FROM sqlite_master WHERE type='table';" # reads the names of all available tables

def drop_columns_from_df(df, columns_to_drop):
    """
    Drops specified columns from a table (pd dataframe).

    Args:
        df (obj): Dataframe object.
        columns_to_drop (list): List of column names to drop.
    """
    try:
        # Drop the specified columns
        df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

        # database_file = conn.__getattribute__(__file__)
        # print(f"Columns {columns_to_drop} dropped from table {table_name} in {database_file}")
        print(f"Columns {columns_to_drop} dropped from table")

        return df

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except pd.io.sql.DatabaseError as e:
        print(f"Pandas Database error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


# CONNECT

In [40]:
# Connect to the .db File

try:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    print(f"Connected to database: {DB_PATH}")
except sqlite3.Error as e:
    print(f"Database connrction error: {e}")
    

Connected to database: datasrc/cac-data-contd-32.db


## Test connection

In [41]:
# List Tables in the Database
tables = pd.read_sql_query(query_read_all_tables_names, conn)
print("Tables in the database:")
# display(tables)
print(tables)
# tables.head()

Tables in the database:
              name
0    organizations
1  sqlite_sequence
2       affiliates


# EXTRACT

In [42]:
# Query Data from a Table
table_name = 'affiliates'
query = f"SELECT * FROM {table_name} LIMIT 10;"
data = pd.read_sql_query(query, conn)
data.head()

,entry_id,organization_id,id,surname,firstname,otherName,email,phoneNumber,gender,formerNationality,...,nationality,address,streetNumber,isChairman,isDesignated,postcode,formerNameType,affiliatesResidentialAddress,affiliatesPscInformation,isPublicUser
0,1,3062361,2395069,ODOH,. E. EMEKA,,emyking08@yahoo.com,08033873502,None,None,...,None,"NO 8 ACCRA ST.WUSE ZONE 5,ABUJA",None,None,None,None,None,None,None,None
1,2,3062361,2395165,OBODOZIE,KENNETH,CHUKWUEBUKA,,,Male,,...,Nigerian,"NO. 12, ROAD 18, UPPER NORTH TRANS EKULU",None,None,None,None,None,None,None,None
2,3,3062361,2395226,OKEFUNA,ANN,ELIZABETH,,,Female,,...,British,"1, OLD DISTILLERY, DINGWALL ROSS-SHINE, IV15 9...",None,None,None,None,None,None,None,None
3,4,3062361,2395278,OBODOZIE,KENNETH,CHUKWUEBUKA,,,Male,,...,Nigerian,"NO. 12, ROAD 18, UPPER NORTH TRANS EKULU",None,None,None,None,None,None,None,None
4,5,3062361,2395279,OKEFUNA,ANN,ELIZABETH,,,Female,,...,British,"1, OLD DISTILLERY, DINGWALL ROSS-SHINE, IV15 9...",None,None,None,None,None,None,None,None


In [ ]:
# Display Query Results with Pandas
data.info()
data.describe()

# TRANSFORM

In [43]:

# transformation 
new_table = data.copy()  # Start with existing data

# new_table['new_column'] = new_table['existing_column'].apply(some_function)
# new_table = new_table.groupby('column').agg({'col1': 'sum', 'col2': 'mean'})

print("Shape of new table:", new_table.shape)
# new_table.head()


Shape of new table: (10, 43)


In [ ]:

# drop column from table

table_name = 'organizations' 
columns_to_drop = ['registrationApproved', 'objectives','registrationSubmitted','paymentDate','...','rrr','timeTakeTobeProcessed','durationInQueue',
                   'resolved','enteredBy','activeForPostInc','consentCode']
affiliate_clm_to_drop = ['isChairman',	'isDesignated',	'postcode',
                         'isCorporate','isPublicUser',]
drop_columns_from_df(new_table, affiliate_clm_to_drop)
new_table.head(4)
# new_table.columns.to_list()


In [52]:
# Join by organisation ID


def join_db_tables(db_path, left_table, right_table, left_on, right_on):
    """
    Connects to a SQLite database, performs an INNER JOIN on two tables,
    and returns the result as a pandas DataFrame.

    Args:
        db_path (str): The file path to the SQLite database.
        left_table (str): The name of the left table in the join.
        right_table (str): The name of the right table in the join.
        left_on (str): The name of the key column in the left table.
        right_on (str): The name of the key column in the right table.

    Returns:
        pandas.DataFrame: A DataFrame containing the joined data,
                          or None if an error occurs.
    """
    joined_df = None
    conn = None
    try:
        # Establish a connection to the SQLite database
        conn = sqlite3.connect(db_path)

        # Construct the SQL INNER JOIN query.
        # Using aliases (lt, rt) for tables makes the query cleaner.
        query = f"""
        SELECT *
        FROM {left_table} AS lt
        INNER JOIN {right_table} AS rt ON lt.{left_on} = rt.{right_on}
        """

        print(f"Executing query:\n{query}")

        # Use pandas to execute the query and load the result into a DataFrame
        joined_df = pd.read_sql_query(query, conn)

        print(f"\nSuccessfully joined '{left_table}' and '{right_table}'.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Ensure the database connection is always closed
        if conn:
            conn.close()

    return joined_df

# --- Example: Create a dummy database to demonstrate the join ---

def setup_dummy_db(db_path="join_example.db"):
    """Creates a dummy database with two tables to be joined."""
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute("DROP TABLE IF EXISTS employees")
        c.execute("DROP TABLE IF EXISTS departments")
        c.execute('''
            CREATE TABLE departments (
                department_id INTEGER PRIMARY KEY,
                department_name TEXT NOT NULL
            )
        ''')
        c.execute('''
            CREATE TABLE employees (
                employee_id INTEGER PRIMARY KEY,
                employee_name TEXT NOT NULL,
                dept_id INTEGER
            )
        ''')
        c.execute("INSERT INTO departments VALUES (101, 'Engineering'), (102, 'Sales')")
        c.execute("INSERT INTO employees VALUES (1, 'Alice', 101), (2, 'Bob', 101), (3, 'Charlie', 102)")
        print(f"Dummy database '{db_path}' created for demonstration.")

# 1. Setup the database (this only needs to be done once)
DUMMY_DB_PATH = 'join_example.db'
setup_dummy_db(DUMMY_DB_PATH)

# 2. Define your database, tables, and join keys
#    (replace these with your actual values)
DB_PATH = DUMMY_DB_PATH
LEFT_TABLE = 'employees'
RIGHT_TABLE = 'departments'
LEFT_KEY = 'dept_id'
RIGHT_KEY = 'department_id'

# 3. Call the function to perform the join
joined_table = join_db_tables(DB_PATH, LEFT_TABLE, RIGHT_TABLE, LEFT_KEY, RIGHT_KEY)

# 4. Display the first few rows of the joined table
if joined_table is not None:
    print("\nJoined table preview:")
    # In a Jupyter environment, display() gives a nicer HTML output
    try:
        display(joined_table.head())
    except NameError:
        print(joined_table.head())


Dummy database 'join_example.db' created for demonstration.
Executing query:

        SELECT *
        FROM employees AS lt
        INNER JOIN departments AS rt ON lt.dept_id = rt.department_id
        

Successfully joined 'employees' and 'departments'.

Joined table preview:


,employee_id,employee_name,dept_id,department_id,department_name
0,1,Alice,101,101,Engineering
1,2,Bob,101,101,Engineering
2,3,Charlie,102,102,Sales


# LOAD

In [ ]:


# Create a connection to the new database
new_conn = sqlite3.connect(NEW_DB_PATH)

# Export the table to the new database
table_name_new_db = 'transformed_table_new'
new_table.to_sql(table_name_new_db,
                  new_conn, 
                  if_exists='replace', index=False)

print(f"Table '{table_name_new_db}' has been exported to {NEW_DB_PATH}")


Table 'transformed_table_new' has been exported to new_analysis.db


In [12]:

# Verify the table was created
verification_query = "SELECT name FROM sqlite_master WHERE type='table';"
new_tables = pd.read_sql_query(verification_query, new_conn)
print("\nTables in the new database:")
display(new_tables)




Tables in the new database:


,name
0,transformed_table
1,transformed_table_new


# CLOSE CONNECTION

In [53]:
# Close the connection to the new database
new_conn.close()
# Cleanup: Close all database connections
try:
    conn.close()
    print("All database connections closed.")
except:
    print("Note: Original database connection was already closed.")

All database connections closed.
